## Environment

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import CITY,CONST,PARAM
from utils import converter
from process import encoding,loss,learning_AMP

np.set_printoptions(threshold=np.inf,linewidth=np.inf,suppress=True)
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
# pd.set_option('max_colwidth',None)
# pd.set_option('display.width',None)
plt.rcParams['font.sans-serif']=['Times New Roman']
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['text.usetex']=False
torch.set_printoptions(threshold=torch.inf,linewidth=1024)

## Data

In [ ]:
# input for others

AM_=converter.Norm_MinMax(encoding.Adjacency_Matrix()).squeeze(-1)
CD_=converter.Norm_MinMax(encoding.Centroid_Distance()).squeeze(-1)
POI_=converter.Norm_MinMax(encoding.Point_of_Interest(),0)
(FLOW_,flow_amp)=converter.Norm_MinMax(encoding.OD_Flow(),need_amplitude=True)

In [ ]:
# input for mine

TE=encoding.Temporal_Encoding()

AM=converter.Norm_MinMax(encoding.Adjacency_Matrix())
CD=converter.Norm_MinMax(encoding.Centroid_Distance())
RW=converter.Norm_MinMax(encoding.Random_Walk())
POI=torch.concatenate((torch.stack([POI_ for i in range(len(POI_))],0),torch.stack([POI_ for i in range(len(POI_))],1)),-1)

GLOBAL=torch.concatenate([TE],-1)
EDGE=torch.concatenate([AM,CD,RW,POI],-1)
FLOW=FLOW_

## Dataset

In [ ]:
offset_mine=PARAM.NUM_HISTORY
offset_others=max(PARAM.W_D_P[0]*CONST.NUM_SLOTs_week,PARAM.W_D_P[1]*CONST.NUM_SLOTs_day,PARAM.W_D_P[2])

In [ ]:
class Dataset_mine(torch.utils.data.Dataset):
    def __init__(self,GLOBAL,EDGE,FLOW,is_train):
        super().__init__()
        self.GLOBAL=GLOBAL
        self.EDGE=EDGE
        self.FLOW=FLOW
        
        self.is_train=is_train
        self.num_history=offset_mine
        self.base=len(FLOW)-offset_mine-(PARAM.NUM_STEPs-1)
        self.copy={'NYC':6,'NB':8}[CITY.NAME]
        
    def __len__(self):
        return self.base*self.copy if self.is_train else self.base
    
    def __getitem__(self,i):
        n=i%self.base+self.num_history
        return ((self.GLOBAL[n],self.EDGE,self.FLOW[n-self.num_history:n]),
                self.FLOW[n:n+PARAM.NUM_STEPs])


class Dataset_others(torch.utils.data.Dataset):
    def __init__(self,AM_,CD_,POI_,FLOW_):
        super().__init__()
        self.AM_=AM_
        self.CD_=CD_
        self.POI_=POI_
        self.FLOW_=FLOW_
    
    def __len__(self):
        return len(self.FLOW_)-offset_others
    
    def __getitem__(self,i):
        n=i+offset_others
        return ((self.AM_,self.CD_,self.POI_,
                 self.FLOW_[[n-i*CONST.NUM_SLOTs_week for i in range(PARAM.W_D_P[0],0,-1)]+\
                            [n-i*CONST.NUM_SLOTs_day for i in range(PARAM.W_D_P[1],0,-1)]+\
                            [n-i for i in range(PARAM.W_D_P[2],0,-1)]]),
                self.FLOW_[n])

In [ ]:
dataset_mine={
    'train':Dataset_mine(GLOBAL[PARAM.START:PARAM.TV_SPLIT],EDGE,FLOW[PARAM.START:PARAM.TV_SPLIT],True),
    'validate':Dataset_mine(GLOBAL[PARAM.TV_SPLIT-offset_mine:PARAM.VT_SPLIT],EDGE,FLOW[PARAM.TV_SPLIT-offset_mine:PARAM.VT_SPLIT],False),
    'test':Dataset_mine(GLOBAL[PARAM.VT_SPLIT-offset_mine:PARAM.END],EDGE,FLOW[PARAM.VT_SPLIT-offset_mine:PARAM.END],False)
}
dataset_others={
    'train':Dataset_others(AM_,CD_,POI_,FLOW_[PARAM.START:PARAM.TV_SPLIT]),
    'validate':Dataset_others(AM_,CD_,POI_,FLOW_[PARAM.TV_SPLIT-offset_others:PARAM.VT_SPLIT]),
    'test':Dataset_others(AM_,CD_,POI_,FLOW_[PARAM.VT_SPLIT-offset_others:PARAM.END])
}

## Learning

In [ ]:
print(CITY.NAME)

### Mine_multistep

In [ ]:
dim_global_in=dataset_mine['train'][0][0][0].shape[-1]
dim_edge_in=dataset_mine['train'][0][0][1].shape[-1]

In [ ]:
from model import Mine_multistep

model_mine=Mine_multistep.Mine(dim_global_in,dim_edge_in)
optimizer_mine=torch.optim.Adam(model_mine.parameters(),{'NYC':4e-4,'NB':3e-4}[CITY.NAME])
scheduler_mine=torch.optim.lr_scheduler.ExponentialLR(optimizer_mine,0.99)
scaler_mine=torch.cuda.amp.GradScaler()

learning_AMP.Loop(model_mine,optimizer_mine,scheduler_mine,scaler_mine,
                  dataset_mine,torch.nn.MSELoss(),loss.Metrics(flow_amp),
                  batch_size={'NYC':64+(0*1086),'NB':48+(0*358)}[CITY.NAME],
                  num_epochs=200,num_workers=0,k=100,load_checkpoint=False)


### GEML

In [ ]:
# from model import GEML

# model_geml=GEML.GEML(AM_,POI_)
# optimizer_geml=torch.optim.Adam(model_geml.parameters(),3e-3)
# scheduler_geml=torch.optim.lr_scheduler.ExponentialLR(optimizer_geml,0.975)

# learning.Loop(model_geml,optimizer_geml,scheduler_geml,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               k=10000)


### MPGCN

In [ ]:
# from model import MPGCN

# model_mpgcn=MPGCN.MPGCN(AM_,POI_)
# optimizer_mpgcn=torch.optim.Adam(model_mpgcn.parameters(),3e-3)
# scheduler_mpgcn=torch.optim.lr_scheduler.ExponentialLR(optimizer_mpgcn,0.975)

# learning.Loop(model_mpgcn,optimizer_mpgcn,scheduler_mpgcn,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               k=10000)


### ST_ED_RMGC

In [ ]:
# from model import ST_ED_RMGC

# model_st_ed_rmgc=ST_ED_RMGC.ST_ED_RMGC(AM_,CD_,POI_,FLOW_)
# optimizer_st_ed_rmgc=torch.optim.Adam(model_st_ed_rmgc.parameters(),3e-3)
# scheduler_st_ed_rmgc=torch.optim.lr_scheduler.ExponentialLR(optimizer_st_ed_rmgc,0.975)

# learning.Loop(model_st_ed_rmgc,optimizer_st_ed_rmgc,scheduler_st_ed_rmgc,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               k=300)


### DMGC_GAN

In [ ]:
# from model import DMGC_GAN

# model_G=DMGC_GAN.Generator()
# optimizer_G=torch.optim.Adam(model_G.parameters(),1e-3)
# scheduler_G=torch.optim.lr_scheduler.ExponentialLR(optimizer_G,0.99)

# learning.Loop(model_G,optimizer_G,scheduler_G,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               k=10000,load_checkpoint=True,change_lr=5e-4)


### History

In [ ]:
# from model import History

# print('HAP:',loss.Format_Loss(loss.Metrics(flow_amp)(*History.HAP(FLOW_))))
# print('HAD:',loss.Format_Loss(loss.Metrics(flow_amp)(*History.HAD(FLOW_))))
# print('HAW:',loss.Format_Loss(loss.Metrics(flow_amp)(*History.HAW(FLOW_))))


### LR

In [ ]:
# from model import ML

# print(loss.Format_Loss(loss.Metrics(flow_amp)(*ML.LR(dataset_others))))


### GBR

In [ ]:
# from model import ML

# print(loss.Format_Loss(loss.Metrics(flow_amp)(*ML.GBR(dataset_others))))


### GCN

In [ ]:
# from model import DL

# model_gcn=DL.GCN(AM_)
# optimizer_gcn=torch.optim.Adam(model_gcn.parameters(),3e-3)
# scheduler_gcn=torch.optim.lr_scheduler.ExponentialLR(optimizer_gcn,0.99)

# learning.Loop(model_gcn,optimizer_gcn,scheduler_gcn,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               num_epochs=200,k=10000)


### LSTM

In [ ]:
# from model import DL

# model_lstm=DL.LSTM()
# optimizer_lstm=torch.optim.Adam(model_lstm.parameters(),1e-3)
# scheduler_lstm=torch.optim.lr_scheduler.ExponentialLR(optimizer_lstm,0.99)

# learning.Loop(model_lstm,optimizer_lstm,scheduler_lstm,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               k=10000)


### Spatial_LSTM

In [ ]:
### from model import DL

# model_spatial_lstm=DL.Spatial_LSTM()
# optimizer_spatial_lstm=torch.optim.Adam(model_spatial_lstm.parameters(),1e-3)
# scheduler_spatial_lstm=torch.optim.lr_scheduler.ExponentialLR(optimizer_spatial_lstm,0.975)

# learning.Loop(model_spatial_lstm,optimizer_spatial_lstm,scheduler_spatial_lstm,
#               dataset=dataset_others,loss_fn=torch.nn.MSELoss(),metrics_fn=loss.Metrics(flow_amp),
#               k=10000)


## Results

In [ ]:
import os
if __name__ == "__main__": os.system("/usr/bin/shutdown")